# KR1: Benchmarking
In this section we investigate the difference of adding `const` to a global variable. By default, dataflow technique will not be able to effectively optimize a global variable since the compiler believes that this variable's value and type may change further down the code. 

In [1]:
using BenchmarkTools;

In this case, we declare a global variable `b` to be used in a function that computes the logarithm of base `b`. 

In [2]:
b = 4
function log_base_b(x::Vector{Float64})
    logs = 0.0
    for i in x
        logs = logs + log(b,i)
    end
    return logs
end
t = rand(100000);
@btime log_base_b($t)

  5.338 ms (300000 allocations: 4.58 MiB)


-72560.71906157274

If we declare the global variable to be `const`, such as `b2` below, the code compiles at a shorter time since we declared this variable's type to not change (its value can still change, however).

In [3]:
const b2 = 4
function log_base_b2(x::Vector{Float64})
    logs = 0.0
    for i in x
        logs = logs + log(b2,i)
    end
    return logs
end
@btime log_base_b2($t)

  581.000 μs (0 allocations: 0 bytes)


-72560.71906157274

# KR2: Polynomial (Naive)
Below is the implementation for computing the value of a polynomial given its coefficients `a_n = a[i]`

In [4]:
function poly_naive(x,a...)
    p = zero(x)
    for i = 1:length(a)
        p = p + a[i]*x^(i-1)
    end
    return p
end
f_naive(x) = poly_naive(x, 4,3,6,7,2,7,3,5,3,5,3)

x = 2.5
pol_n = @benchmark f_naive($x)

BenchmarkTools.Trial: 10000 samples with 535 evaluations.
 Range (min … max):  212.710 ns …  3.706 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     221.121 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   229.102 ns ± 82.288 ns  ┊ GC (mean ± σ):  0.24% ± 1.54%

  ▇▆▇█▆▅▄▃▂▁       ▂▁▁▁▁                                       ▂
  ██████████████▇████████▇▇▇▇▇▆▆▅▄▅▄▅▅▄▅▄▄▃▃▁▃▄▁▄▄▁▁▃▁▃▃▄▃▁▃▃▄ █
  213 ns        Histogram: log(frequency) by time       376 ns <

 Memory estimate: 32 bytes, allocs estimate: 2.

# KR3: Horner's Method (Naive)
Since the coefficients are constants, we can manioulate the polynomial calculation by only using these values instead of computing and adding `x`, through Horner's method. 

In [5]:
function poly_horner(x,a...)
    b = zero(x)
    for i = length(a):-1:1
        b = a[i] + b*x
    end
    return b
end
f_horner(x) = poly_horner(x, 4,3,6,7,2,7,3,5,3,5,3)

pol_h = @benchmark f_horner($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  4.100 ns … 12.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.200 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.220 ns ±  0.402 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇  █                                                       ▁
  █▁▁█▁▁▁▇▁▁▁█▁▁▁█▁▁▁▇▁▁▁▇▁▁▁▇▁▁▇▁▁▁▇▁▁▁▇▁▁▁█▁▁▁▇▁▁▁█▁▁▁▇▁▁▅ █
  4.1 ns       Histogram: log(frequency) by time      5.6 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# KR4: Horner's Method (Macro)
Further optimizing Horner's method, we can implement the calculation of the polynomial using its coefficients further earlier in the compiler pipeline, before type inferencing. This can be done be declaring a macro that performs Horner's method

In [6]:
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

f_horner_macro(x) = @horner(x, 4,3,6,7,2,7,3,5,3,5,3)

mac_h = @benchmark f_horner_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.001 ns … 1.300 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.001 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.034 ns ± 0.049 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▂
  0.001 ns       Histogram: frequency by time        0.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# KR5: Compile + Runtime Comparison
If the naive polynomial computation is to run for 24 hours, these are how fast the other implementations will resolve:

In [7]:
f_h = (mean(pol_h.times) / mean(pol_n.times)) * (24*60);
m_h = (mean(mac_h.times) / mean(pol_n.times)) * (24*60);

println("Horner's method (naive): $f_h minutes")
println("Horner's method (macro): $m_h minutes")

Horner's method (naive): 26.527609664402345 minutes
Horner's method (macro): 0.21369669362826296 minutes
